In [2]:
import scipy.io
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
datafile = './data/ex8data1.mat'
data = scipy.io.loadmat(datafile)
X = data['X']
X_val = data['Xval']
y_val = data['yval']
print X_val.shape
print y_val.shape
#print data

(307, 2)
(307, 1)


In [5]:
scatter_1 = go.Scatter(
    x = X[:,0],
    y = X[:,1],
    mode = 'markers'
)
data = [scatter_1]
py.iplot(data)

In [6]:
def guassian(X,mu,sigma):
    m = X.shape[0]
    n = X.shape[1]
    if sigma.ndim == 1:
        sigma = np.diag(sigma)
    det = np.linalg.det(sigma)
    norm = 1 / (np.power((2 * np.pi),n / 2.) * np.sqrt(det))
    exp_term = np.zeros((m,1))
    inv = np.linalg.inv(sigma)
    for i in xrange(m):
        exp_term[i] = np.exp(-0.5 * (X[i] - mu).T.dot(inv).dot(X[i] - mu))
    return norm * exp_term   

In [7]:
def estimate(X,use_multi=False):
    '''
    estimate the parameter mu and sigma while the mu is mean and sigma is variance
    of each features(n in total)
    if use_multi == True
    input: X-> (m,n)
    output : mu:(n * 1),sigma:(n * 1)
    elif use_multi == False
    we use a multi-Gaussian distribution to model it.
    input X-> (m,n)
    output :mu:(n * 1),cov_mat(n * n)
    '''
    m = X.shape[0]
    mu = np.mean(X,axis=0)
    if not use_multi:
        sigma = np.std(X,axis=0) ** 2
    else:
        #cov matrix
        sigma = (1. / m) * (X - mu).T.dot((X - mu))
    return mu,sigma

In [8]:
#estimate(X)
mu,sigma2 = estimate(X,use_multi=True)
#print guassian(X,mu,sigma2)

In [9]:
def plot_contour(use_multi=False):
    mu,sigma2 = estimate(X,use_multi)
    contour_1 = go.Contour(
        x = X[:,0],
        y = X[:,1],
        z = guassian(X,mu,sigma2)
    )
    data = [scatter_1,contour_1]
    return py.iplot(data)

In [10]:
plot_contour(use_multi=True)###怎么这么炫酷...

### Selecting the threshold, ε

In [11]:
def f1_score(ylabel,ypred):
    #P,R = 0.0 , 0.0
    '''
    ylabel is the true label of the data
    the ypred is the predict label
    P means of all the samples I decide to be true what fraction really True
    R means of all the samples really True what fraction I can correctly decide it to be true
    it's a bit tricky
    F1_score = 2 * P * R / (P + R)
    And we should think the label be boolean numpy array
    In [9]: np.sum([True,True,False,False])
    Out[9]: 2
    '''
    P,R = 0,0
    if float(np.sum(ypred)):
        #P = np.sum([int(ylabel[x]) for x in xrange(ylabel.shape[0]) if ylabel[x]]) / float(np.sum(ypred))
        P = np.sum([int(ylabel[x]) for x in xrange(ypred.shape[0]) \
                    if ypred[x]]) / float(np.sum(ypred))
    #print float(np.sum(ypred))
    #print ypred.shape[0]
    #print P
    #print np.sum([int(ylabel[x]) for x in xrange(ylabel.shape[0]) if ypred[x]])
    if float(np.sum(ylabel)):
        R = np.sum([int(ypred[x]) for x in xrange(ylabel.shape[0]) \
                    if ylabel[x]]) / float(np.sum(ylabel))
    #print R
    #print 2*P*R/(P+R)  , P, R
    #print P + R
    #print P * R * 2.

    return 2. * P * R / (P + R) if (P + R) else 0

In [12]:
f1_score(np.array([False,False,True,False,True]),np.array([True,True,True,False,True]))

0.66666666666666663

In [13]:
def selectThreshold(myycv, mypCVs):
    '''
    mypCVs is a list of all probilities calculated by our guassian
    myycv is the real result
    '''
    nsteps = 1000
    epses = np.linspace(np.min(mypCVs),np.max(mypCVs),nsteps)
    #print epses
    besteps,bestf1 = 0,0
    truevec = (myycv == 1).flatten()
    #print truevec
    for eps in epses:
        ypred = mypCVs < eps
        #print ypred
        this_f1 = f1_score(truevec,ypred)
        #print this_f1
        #print this_f1
        if this_f1 > bestf1:
            bestf1 = this_f1
            besteps = eps
    print "Best F1 is %f, best eps is %0.4g."%(bestf1,besteps)
    return bestf1,besteps

In [14]:
pcvs = guassian(X_val,mu,sigma2)
#print pcvs
bestf1,besteps = selectThreshold(y_val,pcvs)
#print y_val

Best F1 is 0.875000, best eps is 9.075e-05.


#### 1.4 High dimensional dataset

In [15]:
datafile = "./data/ex8data2.mat"
data_2 = scipy.io.loadmat(datafile)
X_2 = data_2['X']
X_val_2 = data_2['Xval']
y_val_2 = data_2['yval']
print X_2.shape

(1000, 11)


In [16]:
mu_2,sigma2_2 = estimate(X_2,use_multi=False)

In [17]:
ypred_2 = guassian(X_val_2,mu_2,sigma2_2)
#ypred_2
print ypred_2.shape
print y_val_2.shape

(100, 1)
(100, 1)


In [18]:
bestf1_2,besteps_2 = selectThreshold(y_val_2,ypred_2)

Best F1 is 0.615385, best eps is 1.379e-18.


In [19]:
ypred_train = guassian(X_2,mu_2,sigma2_2)
#print ypred_train
#print ypred_train < besteps_2
ano_num = np.sum(ypred_train < besteps_2)
print ano_num

117


### recommonded system

In [20]:
datafile_3 = './data/ex8_movies.mat'
movie_data = scipy.io.loadmat(datafile_3)
#print movie_data
Y = movie_data['Y']
R = movie_data['R']
print Y.shape
print R.shape
print Y[:5,:5]
print R[:5,:5]

(1682, 943)
(1682, 943)
[[5 4 0 0 4]
 [3 0 0 0 3]
 [4 0 0 0 0]
 [3 0 0 0 0]
 [3 0 0 0 0]]
[[1 1 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [21]:
data_file_4 = './data/ex8_movieParams.mat'
movie_param = scipy.io.loadmat(data_file_4)
X = movie_param['X']
theta = movie_param['Theta']
print X.shape
print theta.shape
nu = int(movie_param['num_users'])
nm = int(movie_param['num_movies'])
nf = int(movie_param['num_features'])
print nu,nm,nf

(1682, 10)
(943, 10)
943 1682 10


In [22]:
def cfcost(Y,R,X,theta,mylambda = 0.):
    m = Y.shape[0]
    u = Y.shape[1]
    #X = np.zeros((u,dim))
    #theta = np.zeros((m,dim))
    J = np.sum(1. / 2 * R * np.square((X.dot(theta.T) - Y)))
    norm = mylambda / 2. * (np.linalg.norm(X) ** 2 + np.linalg.norm(theta) ** 2)
    cost = J + norm
    return cost

In [23]:
cfcost(Y,R,X,theta)

27918.64012454421

In [24]:
def gradient(Y,R,X,theta,mylambda = 0.):
    term1 = (R * X.dot(theta.T) - Y)
    gradx = term1.dot(theta) + 1. / 2 * mylambda * np.linalg.norm(X) ** 2
    gradtheta = term1.T.dot(X) + 1. / 2 * mylambda * np.linalg.norm(theta) ** 2
    return np.concatenate((gradx.flatten(),gradtheta.flatten()))

In [25]:
gradient(Y,R,X,theta).shape

(26250,)

In [26]:
# So, this file has the list of movies and their respective index in the Y vector
# Let's make a list of strings to reference later
movies = []
with open('data/movie_ids.txt') as f:
    for line in f:
        movies.append(' '.join(line.strip('\n').split(' ')[1:]))

# Rather than rate some movies myself, I'll use what was built-in to the homework
# (just so I can check my solutions)
my_ratings = np.zeros((1682,1))
my_ratings[0]   = 4
my_ratings[97]  = 2
my_ratings[6]   = 3
my_ratings[11]  = 5
my_ratings[53]  = 4
my_ratings[63]  = 5
my_ratings[65]  = 3
my_ratings[68]  = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5